In [ ]:
import os, sys
currentdir = os.path.dirname(os.path.realpath("__file__"))
parentdir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath("__file__"))))
sys.path.append(parentdir)

from joblib import Parallel, delayed
import numpy as np
import pickle 
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torchmetrics
import seaborn as sns
import glob
from tqdm import tqdm
from copy import deepcopy
from src.transformer.metrics import AUL, CorrectedMCC, CorrectedBAcc, CorrectedF1
from src.analysis.metrics.cartesian import cartesian_jit

from torchmetrics import MatthewsCorrCoef, F1Score, Accuracy
from sklearn.utils import resample
from scipy.stats import median_abs_deviation as mad
import time
import cmcrameri.cm as cmc
import matplotlib.colors as clr
from matplotlib.ticker import MultipleLocator, LinearLocator, AutoMinorLocator
plt.rcParams["grid.linestyle"] =  ":"
plt.rcParams["axes.edgecolor"] = "gray"
plt.rcParams["axes.linewidth"] = 0.7

plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "Helvetica"

#### Description
This notebook is used to evaluate the *HEXACO* model (aka *personalities*).

In [ ]:
cmap = cmc.batlowS
cmap

In [ ]:
save_path = r"../plots/hexaco/"

In [ ]:
mean = {"rnd": [.169, .177, .185, .174],
        "rnn": [.111, .030, .050, .010],
        "l2v": [.223, .237, .232, .220],
        }
ci_l = {"rnd":  [.151, .157, .166, .156],
        "rnn":  [.111, .030, .048, .006],
        "l2v":  [.198, .217, .210, .200]}

ci_u = {"rnd":  [.188, .198, .205, .193],
        "rnn":  [.121, .040, .059, .012],
        "l2v":  [.251, .258, .255, .240]}

fig, ax = plt.subplots(1,2, figsize=(15,5))
x_tick = np.array([1,2,3,4])
for i, k in enumerate(mean.keys()):
    yerr = np.vstack([np.array(np.abs(np.array(mean[k]) - np.array(ci_l[k]))),
                     np.array(np.abs(np.array(mean[k]) - np.array(ci_u[k])))])
    ax[0].bar(x = x_tick + i*0.2, height = mean[k], yerr=yerr, width=0.2, capsize=5)
ax[0].set_xticks(x_tick + 0.2)
ax[0].set_xticklabels( ["Q1", "Q2", "Q3", "Q4"])
ax[0].set_title("Macro F1-Score per Personality Aspect")
ax[0].set_ylabel("Score")
ax[0].set_xlabel("Aspect")
ax[0].tick_params(axis= "y", which="major", width=1, length = 6, direction="out", color="gray")
ax[0].tick_params(axis= "y", which="minor", width=1, length =3, direction="out", color="gray")
ax[0].tick_params(axis= "x", which="both", width=1 , length =1)
ax[0].yaxis.set_major_locator(MultipleLocator(0.05))
ax[0].yaxis.set_minor_locator(AutoMinorLocator(5))
sns.despine()

mean = {"rnd": [-0.025, -0.013, 0.009, 0.010],
        "rnn": [ 0.106,  0.086, 0.143, 0.096],
        "l2v": [ 0.126,  0.129, 0.229, 0.109],
        }

mad = {"rnd":  [ 0.013,  0.018, 0.017, 0.013],
        "rnn": [ 0.017,  0.017, 0.017, 0.017],
        "l2v": [ 0.018,  0.015, 0.017, 0.018],
        }


x_tick = np.array([1,2,3,4])
for i, k in enumerate(mean.keys()):
    yerr = np.array(mad[k])
    ax[1].bar(x = x_tick + i*0.2, height = mean[k], yerr=yerr, width=0.2, capsize=5)
ax[1].set_xticks(x_tick + 0.2)
ax[1].set_xticklabels( ["Q1", "Q2", "Q3", "Q4"])
ax[1].set_title("Cohen's Quadratic Kappa Score per Personality Aspect")
ax[1].set_ylabel("Score")
ax[1].set_xlabel("Aspect")
ax[1].tick_params(axis= "y", which="major", width=1, length = 6, direction="out", color="gray")
ax[1].tick_params(axis= "y", which="minor", width=1, length =3, direction="out", color="gray")
ax[1].tick_params(axis= "x", which="both", width=1 , length =1)
ax[1].yaxis.set_major_locator(MultipleLocator(0.05))
ax[1].yaxis.set_minor_locator(AutoMinorLocator(5))
ax[1].axhline(0.0, color="gray", linewidth=0.5, linestyle= ":")

sns.despine()

plt.savefig(save_path + "performance.svg", format="svg")

plt.show()